<a href="https://colab.research.google.com/github/khanh22mse23057/IBM_Python4DataScience/blob/main/Analyzing_Historical_Stock_Revenue_Data_and_Building_a_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
 <li>Question 1 - Extracting Tesla Stock Data Using yfinance</li>
   <li>Question 2 - Extracting Tesla Revenue Data Using Webscraping</li>
   <li>Question 3 - Extracting GameStop Stock Data Using yfinance</li>
   <li>Question 4 - Extracting GameStop Revenue Data Using Webscraping</li>
   <li>Question 5 - Tesla Stock and Revenue Dashboard</li>
   <li>Question 6 - GameStop Stock and Revenue Dashboard</li>
   <li>Question 7 - Sharing your Assignment Notebook</li>
    </ul>
<p>
    Estimated Time Needed: <strong>30 min</strong></p>
</div>

<hr>


In [199]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [200]:
import yfinance as yf
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import seaborn as sns
sns.set_theme(style="whitegrid")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px

fig = px.colors.sequential.swatches_continuous()

## Define Graphing Function


We define the function `make_graph` in this section. You should just be concerned with the inputs; you don't need to understand how the function operates. It requires a dataframe containing information about the stock (the dataframe must have Date and Close columns), a dataframe containing information about the revenue (the dataframe must have Date and Revenue columns), and the stock's name.

In [201]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price",  marker_color='MediumPurple'),  row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue",  marker_color='lightcoral'), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($ USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($ USD Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Extracting Tesla Stock Data Using yfinance


To create a ticker object, use the `Ticker` function and enter the ticker symbol of the stock on which we want to extract data. The stock is called Tesla,and the ticker symbol is `TSLA`.

In [202]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [203]:
tesla_data = tesla.history(period="max")

**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.


In [204]:
tesla_data.reset_index(inplace=True)
tesla_data.head(10)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0
5,2010-07-07 00:00:00-04:00,1.093333,1.108667,0.998667,1.053333,103825500,0,0.0
6,2010-07-08 00:00:00-04:00,1.076000,1.168000,1.038000,1.164000,115671000,0,0.0
7,2010-07-09 00:00:00-04:00,1.172000,1.193333,1.103333,1.160000,60759000,0,0.0
8,2010-07-12 00:00:00-04:00,1.196667,1.204667,1.133333,1.136667,33037500,0,0.0
9,2010-07-13 00:00:00-04:00,1.159333,1.242667,1.126667,1.209333,40201500,0,0.0


## Question 2: Extracting Tesla Revenue Data Using Webscraping


Use the `requests` library to download the webpage https://companiesmarketcap.com/tesla/revenue/. Save the text of the response as a variable named `html_data`.


In [205]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
# url = 'https://companiesmarketcap.com/tesla/revenue/'
# url = 'https://ycharts.com/companies/TSLA/revenues'
html_data = requests.get(url).text

In [206]:
# html_data

Parse the html data using `beautiful_soup`.


In [207]:
soup = BeautifulSoup(html_data,'html5lib')

Using beautiful soup extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column.


In [208]:
tables = soup.find_all('table')
for index,table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)


In [209]:
tesla_revenue.isnull().sum()

Date       0
Revenue    0
dtype: int64

In [210]:
tesla_revenue

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757
5,2021-06-30,11958
6,2021-03-31,10389
7,2020-12-31,10744
8,2020-09-30,8771
9,2020-06-30,6036


<details><summary>Click here if you need help removing the dollar sign and comma</summary>

```
    
If you parsed the HTML table by row and column you can use the replace function on the string
    
    revenue = col[1].text.replace("$", "").replace(",", "")
    
If you use the read_html function you can use the replace function on the string representation of the column

    tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

```

</details>


Remove the rows in the dataframe that are empty strings or are NaN in the Revenue column. Print the entire `tesla_revenue` DataFrame to see if you have any.


In [211]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757
5,2021-06-30,11958
6,2021-03-31,10389
7,2020-12-31,10744
8,2020-09-30,8771
9,2020-06-30,6036


<details><summary>Click here if you need help removing the NaN or empty strings</summary>

```
    
If you have NaN in the Revenue column
    
    tesla_revenue.dropna(inplace=True)
    
If you have emtpty string in the Revenue column

    tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

```

</details>


Display the last 10 row of the `tesla_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [224]:
tesla_revenue.tail(n=10)

,Date,Revenue
43,2011-12-31,39
44,2011-09-30,58
45,2011-06-30,58
46,2011-03-31,49
47,2010-12-31,36
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Question 3: Extracting GameStop Stock Data Using yfinance


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [213]:
GameStop = yf.Ticker('GME')

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [214]:
gme_data=GameStop.history(period="max")

**Reset the index** using the `reset_index(inplace=True)` function on the gme_data DataFrame and display the first five rows of the `gme_data` dataframe using the `head` function. Take a screenshot of the results and code from the beginning of Question 3 to the results below.


In [215]:
gme_data.reset_index(inplace=True)
gme_data.head(10)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0
5,2002-02-21 00:00:00-05:00,1.656318,1.670626,1.641169,1.658002,6976800,0.0,0.0
6,2002-02-22 00:00:00-05:00,1.670626,1.670626,1.615920,1.628544,3525600,0.0,0.0
7,2002-02-25 00:00:00-05:00,1.624336,1.653793,1.605821,1.641169,3453600,0.0,0.0
8,2002-02-26 00:00:00-05:00,1.632753,1.658001,1.606662,1.641169,2761600,0.0,0.0
9,2002-02-27 00:00:00-05:00,1.628545,1.629387,1.599088,1.611712,4091200,0.0,0.0


## Question 4: Extracting GameStop Revenue Data Using Webscraping

Use the `requests` library to download the webpage https://companiesmarketcap.com/gamestop/revenue. Save the text of the response as a variable named `html_data`.


In [216]:
url = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'
html_data = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [217]:
soup = BeautifulSoup(html_data,'html5lib')

Using beautiful soup extract the table with `GameStop Quarterly Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column using a method similar to what you did in Question 2.


In [218]:
tables = soup.find_all('table')
for index,table in enumerate(tables):
    if ("GameStop Quarterly Revenue" in str(table)):
        table_index = index
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text.replace("$", "").replace(",", "")
        gme_revenue = gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)



Display the last five rows of the `gme_revenue` dataframe using the `tail` function. Take a screenshot of the results.


In [225]:
gme_revenue.tail(n=10)

,Date,Revenue
45,2011-04-30,2281
46,2011-01-31,3693
47,2010-10-31,1899
48,2010-07-31,1799
49,2010-04-30,2083
50,2010-01-31,3524
51,2009-10-31,1835
52,2009-07-31,1739
53,2009-04-30,1981
54,2009-01-31,3492


## Question 5: Tesla Stock and Revenue Dashboard


Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`


In [220]:
# tesla_revenue

In [221]:
tesla_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0
...,...,...,...,...,...,...,...,...
3106,2022-10-28 00:00:00-04:00,225.399994,228.860001,216.350006,228.520004,69152400,0,0.0
3107,2022-10-31 00:00:00-04:00,226.190002,229.850006,221.940002,227.539993,61554300,0,0.0
3108,2022-11-01 00:00:00-04:00,234.050003,237.399994,227.279999,227.820007,62688800,0,0.0
3109,2022-11-02 00:00:00-04:00,226.039993,227.869995,214.820007,214.979996,62887200,0,0.0


In [222]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Question 6: GameStop Stock and Revenue Dashboard


Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`.


In [223]:
make_graph(gme_data, gme_revenue, 'GameStop')